In [1]:
import json
import sys
import random
sys.path.append('../')
data_path = '../new_dataset/ropes-train-dev-v1.0/'
train_data = json.load(open(data_path +'/train-v1.0.json'))
dev_data = json.load(open(data_path +'/dev-v1.0.json'))

In [7]:
dev_data['data'][0]['paragraphs'][2]

{'background': 'If a nuclear chain reaction is uncontrolled, it produces a lot of energy all at once. This is what happens in an atomic bomb. However, if a nuclear chain reaction is controlled, it produces energy much more slowly. This is what occurs in a nuclear power plant. The reaction is controlled by inserting rods of nonfissioning material into the fissioning material. You can see this in the Figure below . The radiation from the controlled fission is used to heat water and turn it to steam. The steam is under pressure and causes a turbine to spin. The spinning turbine runs a generator, which produces electricity.',
 'situation': 'The city has recently come upon some nuclear material and is wanting to use it for power generation. They are not sure how to implement it safely and have devised several strategies which include an uncontrolled reaction and a controlled reaction of the material. ',
 'qas': [{'question': 'Which strategy is better suited for generating power?',
   'id': 

In [9]:
from src.dataset_processor import GenerationTasks,QuestionGenerationData
import random

In [13]:
def processText(data,with_outcontext = True):
    data_pack =[]
    for datum in data:
        background = datum['background']
        situation = datum['situation']
        qas = datum['qas']
        for qa in qas:
            q= qa['question']
            answer = random.choice(qa['answers'])['text'] if len(qa['answers'])>0 else ''
            question_object_vanilla = QuestionGenerationData(task=GenerationTasks.vanilla_question_gen,
                                                     input_text= background+' '+situation, output_text= q,
                                                     contextual_text="" )
            data_pack.append(question_object_vanilla)
            if len(answer)>0:
                question_object_context = QuestionGenerationData(task=GenerationTasks.context_question_gen,
                                                     input_text= background+' '+situation,  output_text= q,
                                                     contextual_text=answer )
                data_pack.append(question_object_context)
    return data_pack
            

In [15]:
dev_data_obj = processText(dev_data['data'][0]['paragraphs']) 
len(dev_data_obj)

3376

In [16]:
train_data_obj = processText(train_data['data'][0]['paragraphs']) 
len(train_data_obj)

21848

In [ ]:
from dataclass_csv import DataclassReader, dateformat,DataclassWriter

with open("../curated_data/rope_train.csv", "w",encoding='utf-8') as f:
    w = DataclassWriter(f, train_data_obj, QuestionGenerationData)
    w.write()

with open("../curated_data/rope_dev.csv", "w",encoding='utf-8') as f:
    w = DataclassWriter(f, dev_data_obj, QuestionGenerationData)
    w.write()